<center><h1>Demo cách làm vector Doc2vec ko làm luôn vì máy cùi</h1></center>

In [91]:
from gensim.models import Word2Vec
import os
import warnings
from pyvi import ViTokenizer
import re
import numpy as np
warnings.filterwarnings("ignore")
# Load the data
path = os.getcwd() + '/../Data/demo-full.txt'
sentences = open(path, 'r', encoding='utf-8').read()
stop_words = open(os.getcwd() + '/../Data/stop_word/vietnamese-stopwords.txt', 'r', encoding='utf-8').read()
stop_words = stop_words.split()
# sentences[:100]
# with open(path, 'r', encoding='utf-8') as f:
#     sentences = f.readlines()

In [92]:
def sentences_processing(sentences):
    sentences = sentences.lower()
    sentences = ViTokenizer.tokenize(sentences)
    sentences = sentences.split('\n')
    # remove all the special characters
    sentences = [re.sub(r'\W', ' ', sentence) for sentence in sentences]
    # remove all single characters
    sentences = [re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence) for sentence in sentences]
    # remove single characters from the start
    sentences = [re.sub(r'\^[a-zA-Z]\s+', ' ', sentence) for sentence in sentences]
    # remove numbers
    sentences = [re.sub(r'\d+', '', sentence) for sentence in sentences]
    #  remove stop words
    sentences = [[word for word in sentence.split() if word not in stop_words] for sentence in sentences]
    #  remove words with length < 3
    sentences = [[word for word in sentence if len(word) > 2] for sentence in sentences]
    #  remove words with length > 15
    sentences = [[word for word in sentence if len(word) < 15] for sentence in sentences]
    #  remove empty sentences
    sentences = [sentence for sentence in sentences if len(sentence) > 0]
    return sentences
sentences = sentences_processing(sentences)



In [93]:
#  Train the model
skip_gram = Word2Vec(sentences, min_count=1, vector_size=100, window=3, sg=1)

#  get the sentence vector/ lấy vector của câu
def get_sentence_vector(sentence): #  get the sentence vector/ lấy vector của câu
    sentence = [word for word in sentence if word not in stop_words] #  remove stop words/ loại bỏ stop words
    sentence = [word for word in sentence if len(word) > 2] #  remove words with length < 3/ loại bỏ các từ có độ dài < 3
    sentence = [word for word in sentence if len(word) < 15] #  remove words with length > 15/ loại bỏ các từ có độ dài > 15
    # sentence = [word for word in sentence if word in skip_gram.wv.vocab]
    #new gensnim 4.0.0
    sentence = [word for word in sentence if word in list(skip_gram.wv.key_to_index.keys())] #  remove words not in the vocabulary/ loại bỏ các từ không có trong từ điển
    if len(sentence) == 0: #  remove empty sentences/ loại bỏ các câu trống
        return None #  return None if the sentence is empty/ trả về None nếu câu trống 
    return sum([skip_gram.wv[word] for word in sentence]) / len(sentence) #  return the average of all the word vectors in the sentence/ trả về trung bình của tất cả các vector từ trong câu

#  get the similarity between two sentences
def get_similarity(sentence1, sentence2): #  get the similarity between two sentences/ lấy độ tương tự giữa hai câu
    sentence1 = get_sentence_vector(sentence1) #  get the sentence vector/ lấy vector của câu
    sentence2 = get_sentence_vector(sentence2) #  get the sentence vector/ lấy vector của câu
    if sentence1 is None or sentence2 is None: #  return 0 if one of the sentences is empty/ trả về 0 nếu một trong hai câu trống
        return 0 #  return 0 if one of the sentences is empty/ trả về 0 nếu một trong hai câu trống
    return np.dot(sentence1, sentence2) / (np.linalg.norm(sentence1) * np.linalg.norm(sentence2)) #  return the cosine similarity/ trả về độ tương tự cosine 
    # sentence1 = get_sentence_vector(sentence1)
    # sentence2 = get_sentence_vector(sentence2)
    # if sentence1 is None or sentence2 is None:
    #     return 0
    #     # axis 1 is out of bounds for array of dimension 1
    
    # return skip_gram.wv.cosine_similarities(sentence1, sentence2)[0]

#  get the similarity matrix
def get_similarity_matrix(sentences): #  get the similarity matrix/ lấy ma trận độ tương tự
    n = len(sentences) #  get the number of sentences/ lấy số lượng câu
    similarity_matrix = np.zeros((n, n)) #  create a matrix of zeros/ tạo ma trận 0
    for i in range(n): #  loop through the sentences/ lặp qua các câu
        for j in range(n): #  loop through the sentences/ lặp qua các câu
            similarity_matrix[i][j] = get_similarity(sentences[i], sentences[j]) #  get the similarity between the two sentences/ lấy độ tương tự giữa hai câu
    return similarity_matrix #  return the similarity matrix/ trả về ma trận độ tương tự

#  get the similarity matrix
similarity_matrix = get_similarity_matrix(sentences)


In [118]:
#  get the sentence scores/ lấy điểm câu
def get_sentence_scores(similarity_matrix): #  get the sentence scores/ lấy điểm câu
    sentence_scores = np.zeros(similarity_matrix.shape[0]) #  create a vector of zeros/ tạo vector 0
    sentence_scores = np.sum(similarity_matrix, axis=1) #  get the sum of each row/ lấy tổng của mỗi hàng
    return sentence_scores #  return the sentence scores/ trả về điểm câu

#  get the sentence scores/ lấy điểm câu
sentence_scores = get_sentence_scores(similarity_matrix) #  get the sentence scores/ lấy điểm câu

#  get the summary/ lấy tóm tắt

def get_summary(sentences, sentence_scores, top_n=5): #  get the summary/ lấy tóm tắt
    top_sentence_indices = sentence_scores.argsort()[-top_n:][::-1] #  get the indices of the top n sentences/ lấy chỉ số của n câu hàng đầu
    top_sentence_indices.sort() #  sort the indices/ sắp xếp các chỉ số
    summary = [sentences[i] for i in top_sentence_indices] #  get the top n sentences/ lấy n câu hàng đầu

    return ''.join([w + ' ' for s in summary for w in s])  #  return the summary/ trả về tóm tắt

#  get the summary/ lấy tóm tắt
summary = get_summary(sentences, sentence_scores, 5) #  get the summary/ lấy tóm tắt
summary


'thường_trực thành_ủy hà_nội công_văn đồng_ý đề_xuất ban cán_sự đảng ubnd nội_tại báo_cáo bcs chủ_trương đầu_tư dự_án tổ_hợp thương_mại văn_phòng bán phường cầu diễn quận nam liêm hà_nội khu đất xuân hòa quản_lý trao_đổi phóng_viên báo đầu_tư bất_động_sản vấn_đề luật_sư trần đức phượng đoàn luật_sư hcm diễn tiến cổ_phần_hóa công_ty xuân_hòa quá_trình thay tên đổi chủ sử_dụng đất công_ty xuân hòa sang công_ty nhuệ giang đối_với diện_tích xuân hòa quản_lý có_thể khá giống trường_hợp xem thâu_tóm đất doanh_nghiệp nhà_nước cổ_phần_hóa thông_qua đấu_giá luật đất_đai kiểm_toán nhà_nước nêu trường_hợp như_cụ nguyễn thị thật thanh trì hà_nội cảnh chủ đầu_tư thu_hồi đất bồi_thường giấy_tờ đất tốn thật dân gửi đơn tòa_soạn phản_ánh đất_đai giải_quyết thấu tính đạt quy_định pháp_luật hiện_hành tòa cấp phúc_thẩm tỉnh lâm đồng nhận_định hai bản_án sơ_thẩm vi_phạm tố_tụng nội_dung buộc hủy đưa tòa sơ_thẩm_xét_xử nêu bảo_đảm quyền lợi_ích hợp_pháp dương thị cảnh tuổi ngụ đường nguyễn_thị thập phường 

In [94]:
#  Save the model
skip_gram.save('skip_gram.bin')
#  Load the model
skip_gram = Word2Vec.load('skip_gram.bin')
#  get the 
skip_gram.wv['thủ_tướng']


array([ 5.68059564e-04,  1.55570311e-02,  3.00582871e-03, -3.45600373e-03,
        2.58503645e-03, -2.29644459e-02,  6.56370912e-03,  4.59810384e-02,
        6.02862984e-03,  6.62437174e-03, -2.29573827e-02, -2.00491920e-02,
        3.20818555e-03,  7.57543696e-03,  1.31806117e-02, -1.62626524e-02,
       -4.52424353e-03, -2.55763568e-02,  6.85427850e-03, -2.50079557e-02,
        1.48634715e-02,  6.65149838e-03,  6.96866075e-03, -2.40111910e-02,
       -7.21283443e-03,  1.37907779e-02, -2.02631811e-04, -1.95640624e-02,
       -1.36458296e-02,  5.30999945e-03,  9.34937969e-03,  2.83040619e-03,
        1.48273837e-02, -1.24605419e-02, -2.39533302e-03,  1.89499091e-02,
        6.52748113e-03, -1.11439712e-02, -1.41496677e-02, -2.90083587e-02,
        8.27956200e-03, -2.22081337e-02, -5.10681071e-04,  1.14688268e-02,
        9.43624415e-03, -7.19758263e-03,  2.08234508e-03, -3.57544166e-03,
        7.31441705e-03,  1.26361409e-02,  4.20154817e-03, -5.40881930e-03,
       -1.21997138e-02, -

In [95]:
skip_gram.wv.most_similar(positive=['tiền', 'thuế'], negative=['lương'])

[('đường', 0.9853959083557129),
 ('thu', 0.9852155447006226),
 ('thọ', 0.9849787354469299),
 ('xử_phạt', 0.9848214387893677),
 ('thông_tin', 0.98479825258255),
 ('khu', 0.984673261642456),
 ('quỹ', 0.9846349358558655),
 ('vi_phạm', 0.9846035838127136),
 ('hai', 0.9844377636909485),
 ('trả', 0.9843767881393433)]